In [1]:
index = 1
MODIS_path = r"D:\Code\MODIS_S1_S2\dataset\SatelliteImages\test\MODIS\MODIS_" + str(index) + ".npy"
S1_path = r"D:\Code\MODIS_S1_S2\dataset\SatelliteImages\test\S1\S1_" + str(index) + ".npy"
g_S2_path = r"D:\Code\MODIS_S1_S2\output\generated_image\g_S2_" + str(index) + ".npy"
g_S2_tif_path = r"D:\Code\MODIS_S1_S2\output\generated_image\g_S2_" + str(index) + ".tif"
real_S2_tif_path = r"D:\Code\MODIS_S1_S2\output\generated_image\real_S2_" + str(index) + ".tif"
ref_S2_path = r"D:\ENVI\data\NingBo\NB_roi\cropped_data\test\S2\S2_" + str(index) + ".tif"
model_path = r"D:\Code\MODIS_S1_S2\model\pre_train\pre_train_generator_epoch_101.pth"

print(MODIS_path)
print(S1_path)
print(g_S2_path)
print(g_S2_tif_path)
print(real_S2_tif_path)
print(ref_S2_path)
print(model_path)

D:\Code\MODIS_S1_S2\dataset\SatelliteImages\test\MODIS\MODIS_1.npy
D:\Code\MODIS_S1_S2\dataset\SatelliteImages\test\S1\S1_1.npy
D:\Code\MODIS_S1_S2\output\generated_image\g_S2_1.npy
D:\Code\MODIS_S1_S2\output\generated_image\g_S2_1.tif
D:\Code\MODIS_S1_S2\output\generated_image\real_S2_1.tif
D:\ENVI\data\NingBo\NB_roi\cropped_data\test\S2\S2_1.tif
D:\Code\MODIS_S1_S2\model\pre_train\pre_train_generator_epoch_101.pth


In [2]:
import torch
import numpy as np
import sys
sys.path.append(r"D:\Code\MODIS_S1_S2\src")
from model import Generator

def generate_image(generator, MODIS, S1, save_path=None):
    S2_min = np.array([87, 210, 129, 250, 99, 54, 61, 46])[:, np.newaxis, np.newaxis]
    S2_max = np.array([6884.04, 6996.04, 6944, 7146, 8000.08, 8464, 7320, 7938])[:, np.newaxis, np.newaxis]
    
    generator.eval()
    with torch.no_grad():
        g_S2 = generator(MODIS, S1).cpu().numpy()
    g_S2 = np.squeeze(g_S2)
    g_S2 = (g_S2 + 1) / 2
    g_S2 = (S2_max - S2_min) * g_S2 + S2_min
    g_S2 = np.clip(g_S2, 0, 10000)

    if save_path is not None:
        np.save(save_path, g_S2)
    return g_S2

MODIS_input = torch.from_numpy(np.reshape(np.load(MODIS_path).astype('float32'), (-1, 6, 5, 5))).cuda()
S1_input = torch.from_numpy(np.reshape(np.load(S1_path).astype('float32'), (-1, 2, 250, 250))).cuda()
generator = torch.load(model_path)
g_S2 = generate_image(generator, MODIS_input, S1_input, g_S2_path)

print(g_S2.shape)

(8, 250, 250)


In [2]:
from osgeo import gdal
import numpy as np

def select_bands_from_tif(tif_path, output_path, selected_bands):
    input_ds = gdal.Open(tif_path)

    if input_ds is None:
        print("无法打开输入的TIFF文件")
        return

    # 获取地理转换信息和投影信息
    geotransform = input_ds.GetGeoTransform()
    projection = input_ds.GetProjection()

    # 获取波段数
    num_bands = input_ds.RasterCount

    # 确保选择的波段在有效范围内
    for band in selected_bands:
        if band < 1 or band > num_bands:
            print(f"选择的波段 {band} 超出范围")
            return

    # 创建输出TIFF文件
    driver = gdal.GetDriverByName("GTiff")
    output_ds = driver.Create(output_path, input_ds.RasterXSize, input_ds.RasterYSize, len(selected_bands), gdal.GDT_Float32)

    # 将地理转换信息和投影信息写入输出文件
    output_ds.SetGeoTransform(geotransform)
    output_ds.SetProjection(projection)

    # 将选择的波段写入输出文件
    for idx, band in enumerate(selected_bands, start=1):
        selected_band_data = input_ds.GetRasterBand(band).ReadAsArray()
        output_ds.GetRasterBand(idx).WriteArray(selected_band_data)

    # 关闭数据集
    input_ds = None
    output_ds = None

def save_numpy_as_geotiff(numpy_path, reference_tif_path, output_tif_path):
    numpy_array = np.load(numpy_path)

    # 读取参考tif文件获取地理信息
    reference_dataset = gdal.Open(reference_tif_path)
    reference_geotransform = reference_dataset.GetGeoTransform()
    reference_projection = reference_dataset.GetProjection()

    # 创建输出GeoTIFF文件
    driver = gdal.GetDriverByName("GTiff")
    num_bands, rows, cols = numpy_array.shape
    output_dataset = driver.Create(output_tif_path, cols, rows, num_bands, gdal.GDT_Int16)

    # 设置地理信息
    output_dataset.SetGeoTransform(reference_geotransform)
    output_dataset.SetProjection(reference_projection)

    # 写入数据
    for band_num in range(num_bands):
        output_band = output_dataset.GetRasterBand(band_num + 1)
        output_band.WriteArray(numpy_array[band_num, :, :])

    # 关闭数据集
    output_dataset = None
    reference_dataset = None

select_bands_from_tif(ref_S2_path, real_S2_tif_path, selected_bands=[1, 2, 3, 4, 7, 8, 10, 11])
save_numpy_as_geotiff(g_S2_path, ref_S2_path, g_S2_tif_path)

In [1]:
import matplotlib.pyplot as plt
import numpy as np


def show_image(image):

    rgb = image[:3, :, :]
    rgb = np.transpose(rgb, (1, 2, 0))

    rgb = (rgb-np.min(rgb)) / (np.max(rgb) - np.min(rgb))

    plt.imshow(rgb)
    plt.show()